# Timetables - Explorations

In [25]:
from backend.connect_to_api import ResRobot
from backend.trips import TripPlanner

In [26]:
resrobot= ResRobot()

In [27]:
resrobot.access_id_from_location("Karlstad")

Name                                               extId
KARLSTAD                                           740098025
Karlstad Centralstation                            740000070
Karlstad Busstationen                              740010760
Karlstad, Älvdalsgatan                             740073274
Karlstad, Bergvik Coop                             740022229
Karlstad, Bergvik Maxi                             740057556
Karlstad, Blåljusbyn                               740057772
Karlstad, Brigadmuseet                             740057652
Karlstad, Duettgatan                               740030407
Karlstad, Fadderortsgatan                          740024599


In [28]:
karlstad_bergvik_id = 740022229

## Departures

In [29]:
raw_data = resrobot.timetable_departure(karlstad_bergvik_id)

In [30]:
raw_data

{'Departure': [{'JourneyDetailRef': {'ref': '1|164655|0|1|4022025'},
   'JourneyStatus': 'P',
   'ProductAtStop': {'icon': {'res': 'prod_gen'},
    'operatorInfo': {'name': 'Värmlandstrafik stadstrafik',
     'nameS': '601',
     'nameN': '601',
     'nameL': 'Värmlandstrafik stadstrafik',
     'id': '601'},
    'name': 'Länstrafik - Buss 1',
    'internalName': 'Länstrafik - Buss 1',
    'displayNumber': '1',
    'num': '1',
    'line': '1',
    'lineId': '1601000100001',
    'catOut': 'BLT',
    'catIn': 'BLT',
    'catCode': '7',
    'cls': '128',
    'catOutS': 'BLT',
    'catOutL': 'Länstrafik - Buss',
    'operatorCode': '601',
    'operator': 'Värmlandstrafik stadstrafik',
    'admin': '601___',
    'matchId': '1;1;110'},
   'Product': [{'icon': {'res': 'prod_gen'},
     'operatorInfo': {'name': 'Värmlandstrafik stadstrafik',
      'nameS': '601',
      'nameN': '601',
      'nameL': 'Värmlandstrafik stadstrafik',
      'id': '601'},
     'name': 'Länstrafik - Buss 1',
     'int

In [31]:
import json

# easier reading
print(json.dumps(raw_data, indent=5, ensure_ascii=False))

{
     "Departure": [
          {
               "JourneyDetailRef": {
                    "ref": "1|164655|0|1|4022025"
               },
               "JourneyStatus": "P",
               "ProductAtStop": {
                    "icon": {
                         "res": "prod_gen"
                    },
                    "operatorInfo": {
                         "name": "Värmlandstrafik stadstrafik",
                         "nameS": "601",
                         "nameN": "601",
                         "nameL": "Värmlandstrafik stadstrafik",
                         "id": "601"
                    },
                    "name": "Länstrafik - Buss 1",
                    "internalName": "Länstrafik - Buss 1",
                    "displayNumber": "1",
                    "num": "1",
                    "line": "1",
                    "lineId": "1601000100001",
                    "catOut": "BLT",
                    "catIn": "BLT",
                    "catCode": "7",
            

In [32]:
departures = raw_data.get("Departure", [])

for dep in departures:
    line = dep["ProductAtStop"]["name"]
    direction = dep["direction"]
    stop = dep["stop"]
    time = dep["time"]
    date = dep["date"]
    
    print(f"Line {line} from {stop} to {direction} leaves at {time}, {date}.")


Line Länstrafik - Buss 1 from Karlstad, Bergvik Coop to Karlstad Campus Futurum leaves at 15:07:00, 2025-02-04.
Line Länstrafik - Buss 801 from Karlstad, Bergvik Coop to Slottsbron E45 Åstorpsvägen (Grums kn) leaves at 15:08:00, 2025-02-04.
Line Länstrafik - Buss 801 from Karlstad, Bergvik Coop to Grums Tömtebacken leaves at 15:08:00, 2025-02-04.
Line Länstrafik - Buss 1 from Karlstad, Bergvik Coop to Karlstad, Skutberget Camping leaves at 15:10:00, 2025-02-04.
Line Länstrafik - Buss 1 from Karlstad, Bergvik Coop to Karlstad Campus Futurum leaves at 15:27:00, 2025-02-04.
Line Länstrafik - Buss 1 from Karlstad, Bergvik Coop to Karlstad, Skutberget Camping leaves at 15:30:00, 2025-02-04.
Line Länstrafik - Buss 801 from Karlstad, Bergvik Coop to Segmon (Grums kn) leaves at 15:38:00, 2025-02-04.
Line Länstrafik - Buss 801 from Karlstad, Bergvik Coop to Grums station leaves at 15:38:00, 2025-02-04.
Line Länstrafik - Buss 801 from Karlstad, Bergvik Coop to Karlstad Busstationen leaves at 15:

In [33]:
# exaplring filtering the data, example
target_line = "1"  
target_direction = "Karlstad Campus Futurum" 

# Filter departures
filtered_departures = [
    dep for dep in departures
    if dep["ProductAtStop"]["line"] == target_line and dep["direction"] == target_direction
]

# Print filtered departures
print(f"Next Departures:`´n")
for dep in filtered_departures:
    time = dep["time"]
    date = dep["date"]
    print(f"Line {target_line} to {target_direction} - Departure: {date} {time}")


Next Departures:`´n
Line 1 to Karlstad Campus Futurum - Departure: 2025-02-04 15:07:00
Line 1 to Karlstad Campus Futurum - Departure: 2025-02-04 15:27:00
Line 1 to Karlstad Campus Futurum - Departure: 2025-02-04 15:46:00


In [34]:
# implementing Pandas
import pandas as pd

# Create a DataFrame
df = pd.DataFrame(filtered_departures, columns=["date", "time", "direction"])

# Display DataFrame
print(df)


         date      time                direction
0  2025-02-04  15:07:00  Karlstad Campus Futurum
1  2025-02-04  15:27:00  Karlstad Campus Futurum
2  2025-02-04  15:46:00  Karlstad Campus Futurum


### DataFrame per Stop

In [35]:
departures = raw_data.get("Departure", [])


departure_list = []
for dep in departures:
    line = dep["ProductAtStop"]["line"]
    direction = dep["direction"]
    stop = dep["stop"]
    time = dep["time"]
    date = dep["date"]
    
    departure_list.append({"Time": time, "Direction": direction, "Line": line})


df = pd.DataFrame(departure_list)


if departures:
    stop_name = departures[0]["stop"]
    date = departures[0]["date"]
else:
    stop_name = "Unknown Stop"
    date = "Unknown Date"


print(f"\nNext Departures from {stop_name}")
print(f"{date}\n")
df



Next Departures from Karlstad, Bergvik Coop
2025-02-04



,Time,Direction,Line
0,15:07:00,Karlstad Campus Futurum,1
1,15:08:00,Slottsbron E45 Åstorpsvägen (Grums kn),801
2,15:08:00,Grums Tömtebacken,801
3,15:10:00,"Karlstad, Skutberget Camping",1
4,15:27:00,Karlstad Campus Futurum,1
5,15:30:00,"Karlstad, Skutberget Camping",1
6,15:38:00,Segmon (Grums kn),801
7,15:38:00,Grums station,801
8,15:45:00,Karlstad Busstationen,801
9,15:46:00,Karlstad Campus Futurum,1


In [36]:
# modified to display "Departures in" instead of departure time

from datetime import datetime

now = datetime.now()

df["Departures in"] = df["Time"].apply(
    lambda t: (
        datetime.strptime(f"{date} {t}", "%Y-%m-%d %H:%M:%S") - now
    ).total_seconds()
    // 60
)

# Format Departures in column to display time in hours and minutes, or "now" if minutes == 0
df["Departures in"] = df["Departures in"].apply(
    lambda m: (
        "now"
        if m == 0
        else f"{int(m // 60)} h {int(m % 60)} min" if m >= 60 else f"{int(m)} min"
    )
)

# Drop the original Time column and reorder
df = df.drop(columns=["Time"])
df = df[["Departures in", "Direction", "Line"]]

print(f"\nNext Departures from {stop_name}")
print(f"{date}\n")
df


Next Departures from Karlstad, Bergvik Coop
2025-02-04



,Departures in,Direction,Line
0,6 min,Karlstad Campus Futurum,1
1,7 min,Slottsbron E45 Åstorpsvägen (Grums kn),801
2,7 min,Grums Tömtebacken,801
3,9 min,"Karlstad, Skutberget Camping",1
4,26 min,Karlstad Campus Futurum,1
5,29 min,"Karlstad, Skutberget Camping",1
6,37 min,Segmon (Grums kn),801
7,37 min,Grums station,801
8,44 min,Karlstad Busstationen,801
9,45 min,Karlstad Campus Futurum,1


## Arrivals

In [37]:
raw_data = resrobot.timetable_arrival(karlstad_bergvik_id)

In [38]:
print(json.dumps(raw_data, indent=5))

{
     "Arrival": [
          {
               "JourneyDetailRef": {
                    "ref": "1|164655|0|1|4022025"
               },
               "JourneyStatus": "P",
               "ProductAtStop": {
                    "icon": {
                         "res": "prod_gen"
                    },
                    "operatorInfo": {
                         "name": "V\u00e4rmlandstrafik stadstrafik",
                         "nameS": "601",
                         "nameN": "601",
                         "nameL": "V\u00e4rmlandstrafik stadstrafik",
                         "id": "601"
                    },
                    "name": "L\u00e4nstrafik - Buss 1",
                    "internalName": "L\u00e4nstrafik - Buss 1",
                    "displayNumber": "1",
                    "num": "1",
                    "line": "1",
                    "lineId": "1601000100001",
                    "catOut": "BLT",
                    "catIn": "BLT",
                    "catCode":